## Table of Contents
<a id='table_of_contents'></a>
* [Introduction](#introduction)
* [Simple Storage](#simple_storage)
* [Time Varying Storage Characteristics](#time_varying_characteristics)
* [Ratchets](#ratchets)
* [Commodity Consumed on Injection and Withdrawal](#commodity_consumed)
* [Inventory Cost and Loss](#inventory_cost_loss)
* [Terminal Storage Value](#terminal_storage_value)
* [Freq Argument](#freq_argument)
* [Specifying Time Periods](#specifying_time_periods)

<a id='introduction'></a>
### Introduction
Running the storage valuation models in Python first involves creating an instance of the CmdtyStorage type. In order to represent the diverse characteristics of physical storage there are many different ways of creating CmdtyStorage instances. This workbook gives an overview of the different options available.

In [7]:
# Common Imports
from cmdty_storage import CmdtyStorage
import pandas as pd

<a id='simple_storage'></a>
### The Most Simple Storage and Common Arguments
The code below creates the most simple CmdtyStorage instance, with constant injection/withdrawal rates and costs.

In [17]:
storage_simple = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2022-04-01',
    injection_cost = 0.01,
    withdrawal_cost = 0.025,
    min_inventory = 0.0,
    max_inventory = 1500.0,
    max_injection_rate = 25.5,
    max_withdrawal_rate = 30.9
)

The following arguments are mandatory whenever calling the constructor:
* **freq** specifies the maximum granularity at which injection or withdrawal decision are made. For more details see [this](#freq_argument) section below.
* **storage_start** is the first period where the commodity can be injected into, or withdrawn from storage. In this example it is assigned to an instance of string but see [Specifying Time Periods](#specifying_time_periods) for other types that are also valid for this argument, as well as storage_end.
* **storage_end** is the period AFTER the last period where commodity can be injected or withdrawn. In the case where the storage has non-zero terminal value of inventory, it is on this period where the terminal value is evaluated, as described in [this section](#terminal_storage_value).
* **injection_cost** is the cost per volume unit incurred upon increasing inventory by moving commodity into the storage. In the context of gas storage this is known as 'injection'.
* **withdrawal_cost** is the cost per volume unit incurred upon moving commodity out of storage, hence reducing the inventory. For gas storage this is known as 'withdrawal'.

The following storage constraint arguments are mandatory unless the storage is created with [ratchets](#ratchets).
* **min_inventory** is the minimum volume of inventory which must be held in storage at any time. Most commonly this will have a value of zero.
* **max_inventory** is the maximum volume of inventory that can be held in storage.
* **max_injection_rate** is the maximum volume of commodity which can moved into storage in any time period.
* **max_withdrawal_rate** is the maximum volume of commodity which can be taken out of storage in any time period.

<a id='time_varying_characteristics'></a>
### Time Varying Storage Characteristics
Although the last six argument to the CmdtyStorage constructor are used with scalar float values in the example above, Pandas Series can also be passed in to represent time varying storage characteristics for any of these. The code below gives an example of how this can be used to provide piecewise flat values (varying by year) of a three year storage facility. This is relevant where storage is leased for different nominal volumes of storage facility across time. Although not shown below, **min_inventory**, **max_injection_rate** and **max_withdrawal_rate** can also be time-varying using Pandas Series. Here the call to resample('D') and then fillna('pad') on a Pandas Series are used to generate a piecewise flat curve. 

In [34]:
yearly_period_index = pd.PeriodIndex(freq='D', data = ['2021-04-01', '2022-04-01', '2023-04-01', '2024-04-01'])

storage_time_varying = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2024-04-01',
    injection_cost = pd.Series([0.01, 0.011, 0.013, 0.013], yearly_period_index).resample('D').fillna('pad'),
    withdrawal_cost = pd.Series([0.025, 0.026, 0.027, 0.027], yearly_period_index).resample('D').fillna('pad'),
    min_inventory = 0.0,
    max_inventory = pd.Series([1000, 1300, 800, 800], yearly_period_index).resample('D').fillna('pad'),
    max_injection_rate = 25.5,
    max_withdrawal_rate = 30.9
)

<a id='ratchets'></a>
### Ratchets: Inventory Varying Injection and Withdrawal Rates
For physical storage it is often the case that maximum withdrawal rates will increase, and injection rates will decrease as the storage inventory increases. For natural gas, this due to the increased pressure within the storage cavern and is known as "ratchets". The code below gives an example of creating a CmdtyStorage instance with ratchets by assigning to the **constraints** argument. Note that the arguments min_inventory, max_inventory, max_injection_rate and max_withdrawal_rate should not be assigned to when creating a CmdtyStorage instance with ratchets.

In [30]:
storage_with_ratchets = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2022-04-01',
    injection_cost = 0.01,
    withdrawal_cost = 0.025,
    constraints= [
                ('2021-04-01', # For days after 2021-04-01 (inclusive) until 2022-10-01 (exclusive):
                       [
                            (0.0, -150.0, 250.0),    # At min inventory of zero, max withdrawal of 150, max injection 250
                            (2000.0, -200.0, 175.0), # At inventory of 2000, max withdrawal of 200, max injection 175
                            (5000.0, -260.0, 155.0), # At inventory of 5000, max withdrawal of 260, max injection 155
                            (7000.0, -275.0, 132.0), # At max inventory of 7000, max withdrawal of 275, max injection 132
                        ]),
                  ('2022-10-01', # For days after 2022-10-01 (inclusive):
                       [
                            (0.0, -130.0, 260.0),    # At min inventory of zero, max withdrawal of 130, max injection 260
                            (2000.0, -190.0, 190.0), # At inventory of 2000, max withdrawal of 190, max injection 190
                            (5000.0, -230.0, 165.0), # At inventory of 5000, max withdrawal of 230, max injection 165
                            (7000.0, -245.0, 148.0), # At max inventory of 7000, max withdrawal of 245, max injection 148
                        ]),
                 ]
)

The constraint argument is a list (or any other iterable) of tuples, where the first element of the tuple is the time period from where the constraints apply in a piecewise flat fashion. The second element of the tuple as an iterable of tuples, each with three elements:
* The first element is the inventory.
* The second element is the maximum withdrawal rate.
* The third element is the maximum injection rate.

As well as specifying the injection and withdrawal rates, these inner tuples also specify the minimum and maximum inventory. In the above example the minimum inventory is zero, and the maximum is 7000. Where the inventory is between the inventory pillars, linear interpolation is used to determine the maximum injection and withdrawal rates.

<a id='commodity_consumed'></a>
### Commodity Consumed on Injection and Withdrawal
In the case of natural gas storage it is often the case where some gas will be consumed upon injection or withdrawal. This is because the gas is used as fuel to the motors which pump gas into and out of storage. The optional constructor arguments **cmdty_consumed_inject** and **cmdty_consumed_withdraw** can be used to specify the constant or time varying percentage of commodity volume injected or withdrawn which is consumed upon such decisions. The example below shows an example of constant consumption upon injection and time-varying consumption upon withdrawal.


In [31]:
quarterly_index = pd.PeriodIndex(freq='D', data = ['2021-04-01', '2021-07-01', '2021-10-01', '2022-01-01', '2022-04-01'])
cmdty_consumed_withdraw = pd.Series([0.0023, 0.0026, 0.00115, 0.00321, 0.00321], quarterly_index).resample('D').fillna('pad')

storage_cmdty_consumed = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2022-04-01',
    injection_cost = 0.01,
    withdrawal_cost = 0.025,
    min_inventory = 0.0,
    max_inventory = 1500.0,
    max_injection_rate = 25.5,
    max_withdrawal_rate = 30.9,
    cmdty_consumed_inject = 0.003, # 0.3% of volume injected is consumed
    cmdty_consumed_withdraw = cmdty_consumed_withdraw
)

<a id='inventory_cost_loss'></a>
### Inventory Cost and Inventory Loss
There are two other optional arguments to CmdtyStorage constructor which, although probably not applicable to the main use case of natural gas storage, could be useful for other types of storage, potentially battery electricty storage:
* **inventory_loss** is the percentage of inventory which is "lost" in each time period. Hence, even with no injection or withdrawal, the inventory will reduce by the inventory loss multiplied by the inventory.
* **inventory_cost** is cost of holding inventory as a percentage of the inventory.

As with other arguments, these can either be numeric scalars, or Pandas Series in the case of time-varying characteristics. The example below shows constant inventory loss, and time varying inventory cost which changes for each quarter.

In [35]:
quarterly_index = pd.PeriodIndex(freq='D', data = ['2021-04-01', '2021-07-01', '2021-10-01', '2022-01-01', '2022-04-01'])
inventory_cost_series = pd.Series([0.0001, 0.00012, 0.000115, 0.000121, 0.000121], quarterly_index).resample('D').fillna('pad')

storage_inventory_cost_and_loss = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2022-04-01',
    injection_cost = 0.01,
    withdrawal_cost = 0.025,
    min_inventory = 0.0,
    max_inventory = 1500.0,
    max_injection_rate = 25.5,
    max_withdrawal_rate = 30.9,
    inventory_loss = 0.0002, # 0.02% of inventory is lost every day
    inventory_cost = inventory_cost_series
)

<a id='terminal_storage_value'></a>
### Terminal Storage Value
By default a CmdtyStorage instance assumes that the storage must be empty (i.e. zero inventory) by the end period. However, this can be changed using the **terminal_storage_npv** argument, which is a callable (e.g. a function) which maps from the commodity spot price and the inventory on the final period to the value of the storage. The usage of this argument will probably be rare, but could for example be used in the case of a deal which involves leasing a storage facility with an agreement to sell on any inventory at the end. The example below specifies a storage facility where any inventory left on the end period is worth 97% of the spot price.

In [27]:
storage_with_terminal_value = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2022-04-01',
    injection_cost = 0.01,
    withdrawal_cost = 0.025,
    min_inventory = 0.0,
    max_inventory = 1500.0,
    max_injection_rate = 25.5,
    max_withdrawal_rate = 30.9,
    terminal_storage_npv = lambda spot_price, inventory: 0.97 * spot_price * inventory
)

<a id='freq_argument'></a>
### Freq Argument: Nomination Granularity
The **freq** argument in the CmdtyStorage constructor represents the granularity at which storage injection and withdrawal exercise decisions are made and correspond to Pandas frequency strings as described [here](
https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects). For example if the highest granularity at which the final injection or withdrawal decision can be nominated is daily, use 'D'. The value used is also of significance where other arguments to the CmdtyStorage constructor (described above), arguments to the valuation functions, and attributes of the results from valuation functions are of Pandas types Series or DataFrame. In these cases the Series or DataFrame instances with have an index of PeriodIndex type with frequency that corresponds to this frequency string.

The package level dict variable **FREQ_TO_PERIOD_TYPE** contains a mapping between freq parameter values and the underlying managed types used to represent the resulting curve index type, and hence granularity. As such, the keys of **FREQ_TO_PERIOD_TYPE** can be used to determine the set of admissible values for the freq parameter.

In [6]:
from cmdty_storage import FREQ_TO_PERIOD_TYPE
FREQ_TO_PERIOD_TYPE

{'15min': Cmdty.TimePeriodValueTypes.QuarterHour,
 '30min': Cmdty.TimePeriodValueTypes.HalfHour,
 'H': Cmdty.TimePeriodValueTypes.Hour,
 'D': Cmdty.TimePeriodValueTypes.Day,
 'M': Cmdty.TimePeriodValueTypes.Month,
 'Q': Cmdty.TimePeriodValueTypes.Quarter}

<a id='specifying_time_periods'></a>
### Specifying Time Periods
In the examples above strings (instances of str type) are used for arguments **storage_start**, **storage_end** and the ratchet period start. Howevever, it it is also possible to use instances of the following types as well:
* datetime
* date
* pandas.Period

The code below shows an example of using date and pandas.Period.  

In [29]:
from datetime import date
storage_simple_2 = CmdtyStorage(
    freq='D',
    storage_start = date(2021, 4, 1),
    storage_end = pd.Period('2022-04-01', freq='D'),
    injection_cost = 0.01,
    withdrawal_cost = 0.025,
    min_inventory = 0.0,
    max_inventory = 1500.0,
    max_injection_rate = 25.5,
    max_withdrawal_rate = 30.9
)